# Query DB for Emails

In [ ]:
%load_ext autoreload
%autoreload 2

from MySQL_Scripts.modules.acquiring_db_emails import query_database, drop_unsubscribed


#Query the DB for the customer base that you are narrowing down to, and drop unsubscribed emails
mlb = [
    'Angels', 'Astros', 'Athletics', 'Blue Jays', 'Braves', 'Brewers', 'Cardinals', 'Cubs', 'Diamondbacks', 'Dodgers',
    'Giants', 'Indians', 'Mariners', 'Marlins', 'Mets', 'Nationals', 'Orioles', 'Padres', 'Phillies', 'Pirates',
    'Rangers', 'Rays', 'Red Sox', 'Reds', 'Rockies', 'Royals', 'Tigers', 'Twins', 'White Sox', 'Yankees', 'Baseball', 'MLB'
]

# Create the SQL query string
query = f'''
    SELECT * 
    FROM email_prospecting
    WHERE {' OR '.join([f"LOWER(product_name) LIKE '%{x.lower()}%'" for x in mlb])};
'''

# Assuming you have a function query_database defined
df = query_database(query)
df = drop_unsubscribed(df)

#This is then references from a csv in the sends

# Establish Process to Blast

In [ ]:
from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import time
from Sending_Emails.modules.sends import SendMail

EMAIL_ADDRESS_FROM = 'team@customplanet.com'
EMAIL_PASS = imap_password_customplanet
# Specify database connection details
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'
table_name = 'email_history'

def process(SMTP_CONN, EMAIL_ADDRESS_FROM, EMAIL_PASS, df, contact_column, sport, email_campaign_name, global_subject_line, school=None):

    data_list = []
    processed_emails = set()

    #Limit df itterrows to test
    for index, row in df.iterrows():

 
        #Set the time zone to Central Time
        central_time_zone = pytz.timezone('America/Chicago')
        now_central = datetime.now(central_time_zone)
        formatted_date = now_central.strftime("%m/%d/%Y")
        #When running campaigns to non schools
        try:
            school = row['School Name']
        except:
            school = None

        # png_path = row['PNG_PATH']
        contact = row[contact_column]
  
        # Check if the email has already been processed
        if contact in processed_emails:
            print(f"Skipping email to {contact} as it has already been processed.")
            continue

        data = [school, contact, sport, formatted_date]
        data_list.append(data)

        # Mark the email as processed
        processed_emails.add(contact)
        
        SendMail.send(SMTP_CONN, EMAIL_ADDRESS_FROM, EMAIL_PASS, global_subject_line, school, sport, contact)

    
    data_list = pd.DataFrame(data_list, columns=['School', 'Contact', 'Sport', 'Date_Sent'])

    data_list.rename(columns = {'Contact': 'contact_email', 'School': 'school', 'Date_Sent': 'date_sent', 'Sport': 'sport'}, inplace = True)
    data_list['date'] = data_list['date_sent'].astype(str)

    data_list['subject'] = global_subject_line
    data_list['position'] = contact_column
    data_list['from'] = EMAIL_ADDRESS_FROM
    data_list['email_campaign_tag'] = email_campaign_name
    data_list['date'] = now_central.strftime("%Y-%m-%d %H:%M:%S")

    #This is present in case the process breaks it knows where to resume
    data_list.to_csv('output.csv', index=False)

    return(data_list)

# --------------------------------------Running Process--------------------------

#Temporary testing to modify the Athletic Director email to my own
# df.loc[0, 'email'] = '2015samtaylor@gmail.com'
# df.loc[1, 'email'] = 'sammytaylor2006@yahoo.com'
# df.loc[2, 'email'] = 'jerrybons2006@gmail.com'
# df.loc[2, 'email'] = 'samuel.taylor@greendot.org'
# df.loc[3, 'email'] = 'sammtaylor2006@gmail.com'


#Global Subject line references format string in sends.py, and is also the subject line of the email
#Contact_Column is the column coming from df that passes in the email


In [ ]:
df = df[1520:].reset_index(drop = True)


# df = df[50:]
# df = df.reset_index(drop = True)


# Calling the Function to Send Emails Iteratively

In [ ]:
import logging
import os

#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Sender.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)



#The logic of this function works by declaring the email_history variable which is returned from the process function
# This is a dataframe of the emails who it has been sent to. Also is written to output.csv


def blast(df, EMAIL_ADDRESS_FROM, EMAIL_PASS):

    #This connection sustains throughout the email blast loop
    SMTP_CONN = SendMail.get_smtp_connection(EMAIL_ADDRESS_FROM, EMAIL_PASS)

    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break once end_point has surpassed.
    while len(df) >= 1:

        next_50 = SendMail.get_next_50(df)

        new_point = next_50.index[0]
        end_point = next_50.index[-1]   
        print(new_point, end_point) 
        logging.info(f'\n\nStarting email send at index {new_point}, and ending at index {end_point}')

         # Check if the end_point surpasses 1500
        if end_point >= 1500:
            logging.info('Reached the end_point limit of 1500. Exited the loop.')
            print('Reached the end_point limit of 1500. Exited the loop.')
            break
        
        # Process the emails
        email_history = process(SMTP_CONN, EMAIL_ADDRESS_FROM, EMAIL_PASS, next_50, 'email' ,'Baseball', 'Official MLB Jerseys in Stock!!', global_subject_line)
        email_history.to_csv('output.csv', index = False)
        
        #EXAMPLE OF PROCESS FUNC ARGS(df, contact_column, sport, email_campaign_name, global_subject_line=global_subject_line, school=None)
      
        interval_seconds = 30
        # Wait for the specified interval before the next iteration
        try:
            time.sleep(interval_seconds)
            logging.info(f'Sleeping for {interval_seconds} seconds completed')
            print(f'Sleeping for {interval_seconds} seconds')
        except:
            logging.info('Issue with the sleep')
            print('Issue with the sleep')
    
    os.remove('output.csv')


global global_subject_line
global_subject_line = 'Official MLB Jerseys'
school = None
blast(df)

In [ ]:
#At the end of this process remove output.csv.

#Sends 1300 emails in 45 mins
#Limit to 1500 emails per day
#Anything over 2000 the entire account gets locked for 24 hours. 
#Limits to 2000 emails per day
# SMTPDataError: (550, b'5.4.5 Daily user sending limit exceeded. For more information on Gmail\n5.4.5 sending limits go to\n5.4.5  https://support.google.com/a/answer/166852 w4-20020a4ae9e4000000b005914f455774sm848190ooc.34 - gsmtp')